## Lesson 3: Agentic Search

https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/4/agentic-search-tools

In [1]:
import os

from dotenv import find_dotenv, load_dotenv
from tavily import TavilyClient

load_dotenv(find_dotenv())

True

In [2]:
client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

In [3]:
result = client.search("What is in NVIDIA's new Blackwell GPU?", include_answer=True)

In [4]:
print(result["answer"])

The NVIDIA Blackwell B200 GPU is the company's most powerful single-chip GPU, featuring 208 billion transistors. It is designed for AI applications and is claimed to reduce AI inference operating costs and energy consumption by up to 25 times compared to its predecessor, the H100. The Blackwell architecture includes a Decompression Engine and the ability to access large amounts of memory in the NVIDIA Grace CPU at a speed of 900 gigabytes per second. Major organizations like Amazon Web Services, Dell Technologies, Google, Meta, Microsoft, OpenAI, Oracle, Tesla, and xAI are expected to adopt the Blackwell platform.


### Regular search

In [5]:
city = "New York"

query = f"""
    What is the current weather in {city}? Should I travel there today? "weather.com"
"""

In [14]:
import re

import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS

In [15]:
ddg = DDGS()

In [16]:
def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
    except Exception as e:
        print(f"Returning previous results due to exception reaching ddg: {e}")

        return [
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]

    return [i["href"] for i in results]

In [17]:
for i in search(query=query):
    print(i)

https://weather.com/weather/hourbyhour/l/Manhattan+NY?canonicalCityId=fc47c333c5d13e34e34c9fdb6e047ceb70f7891e01bc9e1d574b5f93f58aa76d
https://weather.com/weather/today/l/96f2f84af9a5f5d452eb0574d4e4d8a840c71b05e22264ebdc0056433a642c84
https://weather.com/weather/hourbyhour/l/Queens+NY?canonicalCityId=235cc3407437509560ae96026aacd4c26a032c07961b37c2f4f7bb5fe24d7579
https://weather.com/weather/tenday/l/96f2f84af9a5f5d452eb0574d4e4d8a840c71b05e22264ebdc0056433a642c84


In [18]:
def scrape_weather_info(url):
    """Scrape content from the given url"""

    if not url:
        return "Weather info could not be found"

    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url=url, headers=headers)

    if response.status_code != 200:
        return "Failed to scrape weather info"

    soup = BeautifulSoup(response.text, "html.parser")
    return soup

In [19]:
url = search(query=query)[0]

In [21]:
soup = scrape_weather_info(url=url)

print(f"Website: {url}\n\n")
print(str(soup.body)[:5000])

Website: https://weather.com/weather/hourbyhour/l/Manhattan+NY?canonicalCityId=fc47c333c5d13e34e34c9fdb6e047ceb70f7891e01bc9e1d574b5f93f58aa76d


<body><div class="appWrapper DaybreakLargeScreen LargeScreen lightTheme twcTheme DaybreakLargeScreen--appWrapper--3kKUE gradients--cloudyFoggyDay--R2hTM gradients--cloudyFoggyDay-top--3dKG9" id="appWrapper"><div class="region-meta"><div class="removeIfEmpty" id="WxuHtmlHead-meta-"></div></div><div class="region-topAds regionTopAds DaybreakLargeScreen--regionTopAds--2kcLJ"><div class="removeIfEmpty" id="WxuAd-topAds-53dce052-5465-4609-a555-c3a20ab64ab0"><div class="adWrapper BaseAd--adWrapper--1dWzA BaseAd--card--2G19u BaseAd--hide--3LkRr"><div class="adLabel BaseAd--adLabel--1LMu9">Advertisement</div><div class="ad_module BaseAd--ad_module--DeGCt subs-undefined BaseAd--placeholder--3IQan" id="WX_Hidden"></div></div></div><div class="removeIfEmpty" id="WxuAd-topAds-fe926b10-58bc-448a-ab09-47e692334250"><div class="adWrapper BaseAd--adWrapper--

In [22]:
weather_data = []

for tag in soup.find_all(["h1", "h2", "h3", "p"]):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

weather_data = "\n".join(weather_data)

# Remove all spaces from the combined text
weather_data = re.sub(r"\s+", " ", weather_data)

print(f"Website: {url}\n\n")
print(weather_data)

Website: https://weather.com/weather/hourbyhour/l/Manhattan+NY?canonicalCityId=fc47c333c5d13e34e34c9fdb6e047ceb70f7891e01bc9e1d574b5f93f58aa76d


recents Specialty Forecasts Hourly Weather - Manhattan, NY There is a marginal risk of severe weather today. Rain Thunderstorms possible after 2 pm. Gusty winds and small hail are possible. Thursday, June 6 10 am Mostly Cloudy 11 am Mostly Cloudy 12 pm Cloudy 1 pm Cloudy 2 pm Scattered Thunderstorms 3 pm Isolated Thunderstorms 4 pm Thunderstorms 5 pm Scattered Thunderstorms 6 pm Isolated Thunderstorms 7 pm Scattered Thunderstorms 8 pm Partly Cloudy 9 pm Partly Cloudy 10 pm Partly Cloudy 11 pm Partly Cloudy Friday, June 7 12 am Partly Cloudy 1 am Clear 2 am Clear 3 am Clear 4 am Clear 5 am Mostly Clear 6 am Partly Cloudy 7 am Partly Cloudy 8 am Partly Cloudy 9 am Partly Cloudy 10 am Partly Cloudy 11 am Partly Cloudy 12 pm Mostly Sunny 1 pm Partly Cloudy 2 pm Partly Cloudy 3 pm Partly Cloudy 4 pm Partly Cloudy 5 pm Partly Cloudy 6 pm Partly Clo

### Agentic Search

In [23]:
result = client.search(query, max_results=1)

In [24]:
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'New York', 'region': 'New York', 'country': 'United States of America', 'lat': 40.71, 'lon': -74.01, 'tz_id': 'America/New_York', 'localtime_epoch': 1717681590, 'localtime': '2024-06-06 9:46'}, 'current': {'last_updated_epoch': 1717681500, 'last_updated': '2024-06-06 09:45', 'temp_c': 23.9, 'temp_f': 75.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 250, 'wind_dir': 'WSW', 'pressure_mb': 1001.0, 'pressure_in': 29.56, 'precip_mm': 0.04, 'precip_in': 0.0, 'humidity': 82, 'cloud': 75, 'feelslike_c': 26.1, 'feelslike_f': 78.9, 'windchill_c': 22.8, 'windchill_f': 73.0, 'heatindex_c': 25.0, 'heatindex_f': 77.0, 'dewpoint_c': 20.6, 'dewpoint_f': 69.1, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 5.0, 'gust_mph': 12.9, 'gust_kph': 20.8}}


In [25]:
import json

from pygments import formatters, highlight, lexers

In [26]:
parsed_json = json.loads(data.replace("'", '"'))

formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(
    formatted_json, lexers.JsonLexer(), formatters.TerminalFormatter()
)

In [27]:
print(colorful_json)

{
    "location": {
        "name": "New York",
        "region": "New York",
        "country": "United States of America",
        "lat": 40.71,
        "lon": -74.01,
        "tz_id": "America/New_York",
        "localtime_epoch": 1717681590,
        "localtime": "2024-06-06 9:46"
    },
    "current": {
        "last_updated_epoch": 1717681500,
        "last_updated": "2024-06-06 09:45",
        "temp_c": 23.9,
        "temp_f": 75.0,
        "is_day": 1,
        "condition": {
            "text": "Partly cloudy",
            "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png",
            "code": 1003
        },
        "wind_mph": 3.8,
        "wind_kph": 6.1,
        "wind_degree": 250,
        "wind_dir": "WSW",
        "pressure_mb": 1001.0,
        "pressure_in": 29.56,
        "precip_mm": 0.04,
        "precip_in": 0.0,
        "humidity": 82,
        "cloud": 75,
        "feelslike_c": 26.1,
        "feelslike_f": 78.9,
        "windchill_c": 22.8,
        "windchill_